# TODO

- You need to specifiy and justify the relevant regressions (check Lene abstract)
- you need plots
- You need to figure why you have nans in month_id, and why there is one nan in gid.
- You need to check taht each gid only have t (49 ish) month_ids... # no! you can have that. The images are unit of ananlysis, not the cell. 
    - so exposure also needs to be modeled in the ucdp/prop df


- You need conflict exposure

- you need to do that thing with checking the corr between "person - male" and "female"

In [1]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
from linearmodels.panel import PanelOLS


In [2]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}bodies_df_fatal.pkl', 'rb') as file:
    bodies_df_fatal = pickle.load(file)

# NOTES:

- First of you do not have spatial or temporal exposure here. Se if scikit learn is faster then pymc3
- Second what is the models?

RQ:
To what extent does the distribution of content in the images, differs across publication status.
and
To What extent does the distribution of content in the images corrolate with the magnitude/exposure of conflict 


$$ feature  = \beta_{ij} pubstatus + \theta_{ij} Magnitude + \epsilon $$


So to do this you need a dummy for each publication status really


In [7]:
# pre

list_of_features_mean = [feature for feature in bodies_df_fatal.columns if 'mean' in feature] 

df_sub = bodies_df_fatal[list_of_features_mean + ['log_best', 'publication', 'month_id', 'gid']].copy()
df_sub = df_sub[df_sub['person_mean'] != 0] # no people, no entry
df_sub[['Published', 'Raw', 'Submitted']] = pd.get_dummies(df_sub[['publication']])
df_sub.drop(columns= 'publication', inplace= True)
df_sub.dropna(inplace= True)
df_sub.set_index(['gid', 'month_id'], inplace = True)
df_sub['intPubBest'] = df_sub['Published'] * df_sub['log_best']

# also drop all with no persons in them

## Contiues dependents:

$$
female_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
religiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [8]:
X = df_sub.loc[:,['log_best', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2542
Estimator:,PanelOLS,R-squared (Between):,0.7204
No. Observations:,126548,R-squared (Within):,0.2526
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4840
Time:,11:55:25,Log-likelihood,-1.454e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.078e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.909e+18


In [16]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3404
Estimator:,PanelOLS,R-squared (Between):,0.9104
No. Observations:,126548,R-squared (Within):,0.3517
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4274
Time:,12:03:13,Log-likelihood,-1.74e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.632e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,4.465e+19


### Binary dependent (linar probability model)

$$
binaryFemale_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
binaryFeligiousGarments_{it} =  \gamma_{i} + \alpha_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [10]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = (df_sub.loc[:, 'female_mean'] > 0).astype(int)


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.0758
Estimator:,PanelOLS,R-squared (Between):,0.4731
No. Observations:,126548,R-squared (Within):,0.0772
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.2993
Time:,11:55:29,Log-likelihood,-8.038e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,2593.7
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.646e+17


In [11]:
X = df_sub.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = (df_sub.loc[:, 'religiousGarmentFemale_mean'] > 0).astype(int)

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.2533
Estimator:,PanelOLS,R-squared (Between):,0.8558
No. Observations:,126548,R-squared (Within):,0.2663
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4119
Time:,11:55:31,Log-likelihood,-4.076e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8579.7
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(5,126473)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,2.445e+20


## Logged dependent

$$
logFemale_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
logFeligiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [12]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = np.log(df_sub.loc[:, 'female_mean']+1)


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2046
Estimator:,PanelOLS,R-squared (Between):,0.7023
No. Observations:,126548,R-squared (Within):,0.2032
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4531
Time:,11:55:33,Log-likelihood,-5.737e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8133.4
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,7.965e+19


In [13]:
X = df_sub.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = np.log(df_sub.loc[:, 'religiousGarmentFemale_mean'] + 1)

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3339
Estimator:,PanelOLS,R-squared (Between):,0.9286
No. Observations:,126548,R-squared (Within):,0.3463
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4581
Time:,11:55:35,Log-likelihood,3.722e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.268e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(5,126473)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,9.003e+19


## ratio dependent

$$
ratioFemalePerson_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
ratioFemaleReligiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} + female_{it} \epsilon_{it}  
$$

In [14]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'female_mean']/df_sub.loc[:, 'person_mean']


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,0,R-squared:,0.0070
Estimator:,PanelOLS,R-squared (Between):,-0.1795
No. Observations:,126548,R-squared (Within):,0.0074
Date:,"Mon, Jun 27 2022",R-squared (Overall):,-0.0866
Time:,11:55:37,Log-likelihood,-4.297e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,222.22
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-9.783e+18


In [15]:
df_sub_fem = df_sub[df_sub['female_mean']>0].copy()

X = df_sub_fem.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub_fem.loc[:, 'religiousGarmentFemale_mean']/df_sub_fem.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,0,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,0.2531
No. Observations:,72153,R-squared (Within):,0.0061
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.1782
Time:,11:55:38,Log-likelihood,-1.576e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,161.68
Entities:,23,P-value,0.0000
Avg Obs:,3137.1,Distribution:,"F(5,72082)"
Min Obs:,0.0000,,
Max Obs:,6.138e+04,F-statistic (robust):,7.842e+19


Plots - temporal and spatial

# GPS . do in the udcp/prior df. this one have img as unit of analysis

In [20]:
df_sub['log_best'].std()

1.6336288033999973

In [18]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF


In [ ]:
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

In [21]:
noise_std = df_sub['log_best'].std()
gaussian_process = GaussianProcessRegressor( kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9)

In [69]:
mask = (df_sub.reset_index(level = 0)['gid'] == df_sub[df_sub['log_best'] == df_sub['log_best'].max()].reset_index(level = 0)['gid'].unique().item()).values
df_gp = df_sub[mask]

y = df_gp['log_best']
x = np.array(df_gp.reset_index(level = 1)['month_id']).reshape(-1, 1)

In [70]:
gaussian_process.fit(x, y)
gaussian_process.kernel_

MemoryError: Unable to allocate 79.8 GiB for an array with shape (103500, 103500) and data type float64

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(x, return_std=True)

In [71]:
df_gp

person_mean  child_mean  male_mean  adult_mean  youth_mean  \
gid      month_id                                                               
177569.0 348.0             3.0         0.0        3.4         2.8         0.0   
         357.0             3.0         0.0        1.4         1.2         0.0   
         334.0             1.6         0.0        1.6         2.4         0.0   
         349.0             1.4         0.0        1.2         1.2         0.0   
         281.0             0.8         0.0        0.4         0.6         0.0   
...                        ...         ...        ...         ...         ...   
         325.0             1.8         0.0        0.8         1.0         0.0   
         279.0             4.6         1.5        5.2         5.0         0.5   
         280.0             2.0         1.5        2.0         1.4         1.0   
         280.0             2.0         1.5        2.0         1.4         1.0   
         280.0             2.0         1.5        2.0         1.4         1.0   

                   falgIRQ_mean  female_mean  religiousGarmentFemale_mean  \
gid      month_id                                                           
177569.0 348.0              0.0          0.0                          0.0   
         357.0              0.0          0.0                          0.0   
         334.0              0.0          0.0                          0.0   
         349.0              0.0          0.0                          0.0   
         281.0              0.0          0.0                          0.0   
...                         ...          ...                          ...   
         325.0              0.0          0.0                          0.0   
         279.0              0.0          2.0                          0.0   
         280.0              0.0          1.0                          1.0   
         280.0              0.0          1.0                          1.0   
         280.0              0.0          1.0                          1.0   

                   uniformed_mean  firearm_mean  ...  bloodedArea_mean  \
gid      month_id                                ...                     
177569.0 348.0                0.2           0.0  ...               0.0   
         357.0                2.6           0.0  ...               0.0   
         334.0                0.6           0.0  ...               0.0   
         349.0                0.8           0.0  ...               0.0   
         281.0                0.2           0.0  ...               0.0   
...                           ...           ...  ...               ...   
         325.0                2.0           0.0  ...               0.0   
         279.0                0.2           0.0  ...               0.0   
         280.0                0.0           1.0  ...               0.0   
         280.0                0.0           1.0  ...               0.0   
         280.0                0.0           1.0  ...               0.0   

                   militaryVehicle_mean  hostage_mean  casualty_mean  \
gid      month_id                                                      
177569.0 348.0                      0.0           0.0            0.0   
         357.0                      0.2           0.5            0.0   
         334.0                      0.0           0.0            0.0   
         349.0                      0.0           1.0            0.5   
         281.0                      0.6           0.0            0.0   
...                                 ...           ...            ...   
         325.0                      0.8           0.0            0.0   
         279.0                      0.0           0.0            0.0   
         280.0                      0.0           0.0            0.0   
         280.0                      0.0           0.0            0.0   
         280.0                      0.0           0.0            0.0   

                   elderly_mean  log_best  Published  Raw  Subm

In [ ]:
gaussian_process.fit(X_train, y_train)

# old

In [19]:

fitted_dict = dict()

for i in list_of_features_mean:

    X = df_sub.loc[:,['log_best', 'Published', 'Submitted']] # raw is ref
    y = df_sub.loc[:, i]

    m = PanelOLS(dependent=y,
                exog=X,
                entity_effects=True,
                time_effects=True)

    fitted_dict[i] = m.fit(cov_type='clustered', cluster_entity=True)

In [20]:
fitted_dict

{'person_mean':                           PanelOLS Estimation Summary                           
 Dep. Variable:            person_mean   R-squared:                        0.0006
 Estimator:                   PanelOLS   R-squared (Between):             -0.0585
 No. Observations:              152188   R-squared (Within):              -0.0119
 Date:                Mon, Jun 27 2022   R-squared (Overall):             -0.2251
 Time:                        10:23:22   Log-likelihood                -3.275e+05
 Cov. Estimator:             Clustered                                           
                                         F-statistic:                      31.697
 Entities:                          23   P-value                           0.0000
 Avg Obs:                       6616.9   Distribution:                F(3,152113)
 Min Obs:                       2.0000                                           
 Max Obs:                    1.234e+05   F-statistic (robust):          2.732e+25
 

In [28]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3554
Estimator:,PanelOLS,R-squared (Between):,0.9075
No. Observations:,152188,R-squared (Within):,0.3676
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4309
Time:,10:48:20,Log-likelihood,-7166.9
Cov. Estimator:,Clustered,,
,,F-statistic:,2.097e+04
Entities:,23,P-value,0.0000
Avg Obs:,6616.9,Distribution:,"F(4,152112)"
Min Obs:,2.0000,,
Max Obs:,1.234e+05,F-statistic (robust):,6.135e+21


In [21]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3554
Estimator:,PanelOLS,R-squared (Between):,0.9075
No. Observations:,152188,R-squared (Within):,0.3676
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4309
Time:,10:24:07,Log-likelihood,-7166.9
Cov. Estimator:,Clustered,,
,,F-statistic:,2.097e+04
Entities:,23,P-value,0.0000
Avg Obs:,6616.9,Distribution:,"F(4,152112)"
Min Obs:,2.0000,,
Max Obs:,1.234e+05,F-statistic (robust):,6.135e+21
